In [1]:
%run init.ipynb

matchzoo version 1.0
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [2]:
preprocessor = mz.models.ArcI.get_default_preprocessor(
    filter_mode='df',
    filter_low_freq=2,
)

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 5688.52it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 4144.24it/s]
Building Vocabulary from a datapack.: 100%|██████████| 404432/404432 [00:00<00:00, 2478649.32it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 8181.56it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 4103.55it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 754889.78it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 8076.03it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<00

In [4]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7efff02aa0b8>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7efff069ed30>,
 'vocab_size': 16675,
 'embedding_input_dim': 16675}

In [5]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [6]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed
)

In [7]:
padding_callback = mz.models.ArcI.get_default_padding_callback(
    fixed_length_left=10,
    fixed_length_right=100,
    pad_word_value=0,
    pad_word_mode='pre'
)

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=20,
    stage='train',
    resample=True,
    sort=False,
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=20,
    stage='dev',
    callback=padding_callback
)

In [8]:
model = mz.models.ArcI()

model.params['task'] = ranking_task
model.params['embedding'] = embedding_matrix
model.params['left_length'] = 10
model.params['right_length'] = 100
model.params['left_filters'] = [128]
model.params['left_kernel_sizes'] = [3]
model.params['left_pool_sizes'] = [4]
model.params['right_filters'] = [128]
model.params['right_kernel_sizes'] = [3]
model.params['right_pool_sizes'] = [4]
model.params['conv_activation_func'] = 'relu'
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 100
model.params['mlp_num_fan_out'] = 1
model.params['mlp_activation_func'] = 'relu'
model.params['dropout_rate'] = 0.9

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

ArcI(
  (embedding): Embedding(16675, 300)
  (conv_left): Sequential(
    (0): Sequential(
      (0): ConstantPad1d(padding=(0, 2), value=0)
      (1): Conv1d(300, 128, kernel_size=(3,), stride=(1,))
      (2): ReLU()
      (3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (conv_right): Sequential(
    (0): Sequential(
      (0): ConstantPad1d(padding=(0, 2), value=0)
      (1): Conv1d(300, 128, kernel_size=(3,), stride=(1,))
      (2): ReLU()
      (3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (dropout): Dropout(p=0.9, inplace=False)
  (mlp): Sequential(
    (0): Sequential(
      (0): Linear(in_features=3456, out_features=100, bias=True)
      (1): ReLU()
    )
    (1): Sequential(
      (0): Linear(in_features=100, out_features=1, bias=True)
      (1): ReLU()
    )
  )
  (out): Linear(in_features=1, out_features=1, bias=True)
)
Trainable params:  5578959


In [9]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [10]:
trainer.run()

[Iter-102 Loss-0.950]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4339 - normalized_discounted_cumulative_gain@5(0.0): 0.5039 - mean_average_precision(0.0): 0.4654



[Iter-204 Loss-0.846]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5491 - normalized_discounted_cumulative_gain@5(0.0): 0.6054 - mean_average_precision(0.0): 0.5702



[Iter-306 Loss-0.722]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5355 - normalized_discounted_cumulative_gain@5(0.0): 0.5959 - mean_average_precision(0.0): 0.5558



[Iter-408 Loss-0.668]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5323 - normalized_discounted_cumulative_gain@5(0.0): 0.597 - mean_average_precision(0.0): 0.5522



[Iter-510 Loss-0.590]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5602 - normalized_discounted_cumulative_gain@5(0.0): 0.6138 - mean_average_precision(0.0): 0.5738



[Iter-612 Loss-0.524]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5609 - normalized_discounted_cumulative_gain@5(0.0): 0.6304 - mean_average_precision(0.0): 0.5848



[Iter-714 Loss-0.461]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5742 - normalized_discounted_cumulative_gain@5(0.0): 0.6483 - mean_average_precision(0.0): 0.6031



[Iter-816 Loss-0.364]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5794 - normalized_discounted_cumulative_gain@5(0.0): 0.648 - mean_average_precision(0.0): 0.6049



[Iter-918 Loss-0.298]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5267 - normalized_discounted_cumulative_gain@5(0.0): 0.6109 - mean_average_precision(0.0): 0.5608



[Iter-1020 Loss-0.280]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5961 - normalized_discounted_cumulative_gain@5(0.0): 0.6492 - mean_average_precision(0.0): 0.6111

Cost time: 1278.5495643615723s
